# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame

file = "../output_data/clean_city_data.csv"
file_df = pd.read_csv(file)
file_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Busselton,0,AU,1592754655,60,-33.65,115.33,54.00,14.47
1,1,Carnarvon,0,AU,1592754661,93,-24.87,113.63,60.80,4.85
2,2,La Primavera,43,CO,1592754661,72,5.49,-70.41,84.85,3.02
3,3,Shizunai-furukawachō,50,JP,1592754661,87,42.33,142.37,54.45,10.22
4,4,Rikitea,75,PF,1592754661,70,-23.12,-134.97,71.96,17.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Find the maximum value of Humidity to set the maximum intensity for the heatmap layer
file_df['Humidity'].max()

100

In [7]:
#  Access maps with unique API key
gmaps.configure(api_key=g_key)

# Create an empty list to store the locations
locations = []

# Use the latitude and longitude as locations
file_df['Location'] = list(zip(file_df["Latitude"], file_df["Longitude"]))

# Store latitude and longitude in locations list
for coord in file_df['Location']:
    locations.append(coord)

# To use humidity as the weight    
humidity = file_df['Humidity']

# Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)


# Display figure
fig 



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Narrow down the cities to fit weather conditions

    # A max temperature lower than 80 degrees but higher than 70.

    # Wind speed less than 10 mph.

    # Zero cloudiness.

    # Drop any rows that don't contain all three conditions and any rows will null values.

hotel_df = file_df[(file_df['Max Temp'] < 80) & (file_df['Max Temp'] > 70) & (file_df['Wind Speed'] < 10) & 
                 (file_df['Cloudiness'] == 0)]
hotel_df["Hotel Name"] = ""
hotel_df.dropna(inplace=True)

/Users/dayothompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/dayothompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# set up a parameters dictionary

params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

# Use the latitude and longitude to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

# change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    hotels = requests.get(base_url, params=params)
    hotels_json = hotels.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_json["results"][0]["name"]
    except IndexError:
        print("Name not found")


Name not found
Name not found


/Users/dayothompson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Name not found


In [11]:
# Drop any empty Hotel row in the DataFrame

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

/Users/dayothompson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/dayothompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Location,Hotel Name
189,189,Seoul,0,KR,1592754493,88,37.57,126.98,73.40,2.24,"(37.57, 126.98)",K-POP HOTEL Seoul Tower
212,212,Fukuechō,0,JP,1592754688,84,32.69,128.84,72.81,6.53,"(32.69, 128.84)",GOTO TSUBAKI HOTEL
214,214,Faxinal,0,BR,1592754689,46,-24.00,-51.32,76.32,8.75,"(-24.0, -51.32)","Hotel Matrix R 139,00 individual"
265,265,Phalaborwa,0,ZA,1592754695,20,-23.94,31.14,77.00,1.12,"(-23.94, 31.14)",Kubu Safari Lodge
348,348,Santo Antônio do Sudoeste,0,BR,1592754707,57,-26.07,-53.73,75.04,5.68,"(-26.07, -53.73)",Hotel Titos
493,493,Arganil,0,PT,1592754729,60,40.22,-8.05,78.01,8.01,"(40.22, -8.05)",Hotel Solneve


In [12]:
# NOTE: Do not change any of the code in this cell
# hotel_locations = [hotel for hotel in hotel_df['Hotel Name']]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure

humidity= file_df['Humidity']
location2 = file_df['Location']
# # fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location2, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig 



Figure(layout=FigureLayout(height='420px'))